**Scraping the data from https://docs.nvidia.com/cuda/**

In [ ]:
# !pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://docs.nvidia.com/cuda/"

response = requests.get(url)

if response.status_code == 200:
  soup = BeautifulSoup(response.content, "html.parser")

  h1_elements = soup.find_all("h1")
  h2_elements = soup.find_all("h2")
  p_elements = soup.find_all("p")

  data = []

  for element in h1_elements:
    title = element.text.strip()
    data.append({"h1 title": title})

  for element in h2_elements:
    title = element.text.strip()
    data.append({"h2 title": title})

  for element in p_elements:
    description = element.text.strip()
    data.append({"description": description})

else:
  print("Error: Failed to retrieve the webpage")


In [ ]:
scraped_text = []
for text in data:
    if 'description' in text:
        scraped_text.append(text['description'])

In [ ]:
# scraped_text

**Creating milvus vector store to store the embeddings of the text created by embedding model qwen-1.5B embedding model then insert/store the embeddings into the milvus vector DB collection and then perform semantic search for a query on the vector DB collection to get the relevant result and further used re-ranking algorithm to rank the relevant result**

storing the data into milvus vector database (collection)

In [ ]:
!pip install --upgrade pymilvus
!pip install "pymilvus[model]"
# !pip install -U pymilvus
# !pip install grpcio<=1.63.0


In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 5ff2c6b5da154d9ba776ecff4d3eae8e


In [ ]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
collection = client.create_collection(
    collection_name="demo_collection",
    dimension=1536,  # The vectors we will use in this demo has 1536 dimensions
)


DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: demo_collection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: demo_collection


In [ ]:
from pymilvus import model

sentence_transformer_ef = model.dense.SentenceTransformerEmbeddingFunction(
    model_name="Alibaba-NLP/gte-Qwen2-1.5B-instruct", # Specify the model name
    device='cpu' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
docs_embeddings = sentence_transformer_ef.encode_documents(scraped_text)

print("Embeddings:", docs_embeddings)
print("Dim:", sentence_transformer_ef.dim, docs_embeddings[0].shape)


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Embeddings: [array([ 0.0139805 ,  0.00859524,  0.00332981, ..., -0.0054694 ,
        0.05552983,  0.01226447], dtype=float32), array([ 0.00726299, -0.00881416, -0.01092338, ..., -0.01055073,
        0.05871813,  0.01341031], dtype=float32), array([-0.0115285 ,  0.01403528,  0.01769487, ..., -0.00720611,
        0.06313925,  0.00700546], dtype=float32), array([ 0.00643452, -0.00153371, -0.00726041, ...,  0.01672272,
        0.03427675,  0.02605665], dtype=float32), array([ 0.02180496, -0.02389077, -0.01656532, ...,  0.00178688,
        0.0157576 ,  0.00485616], dtype=float32), array([ 0.01482052,  0.0324157 ,  0.00970147, ..., -0.00695904,
       -0.0063861 ,  0.03182848], dtype=float32), array([ 6.4229262e-03,  2.5869396e-03, -6.1484032e-05, ...,
       -1.8032525e-02,  1.5893860e-02, -2.4495490e-03], dtype=float32), array([ 0.04251689, -0.00274006, -0.01031493, ...,  0.00123716,
        0.0215228 , -0.00732832], dtype=float32), array([ 0.03016854, -0.03855629,  0.00663653, ...,  0.001

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: b62314f4dbfb45be9e20c1dd4fcfbca0


In [ ]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=1536,  # The vectors we will use in this demo has 1536 dimensions
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: demo_collection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: demo_collection


In [ ]:
len(docs_embeddings)

85

In [ ]:
data = [
    {"id": i, "vector": docs_embeddings[i], "text": scraped_text[i], "subject": "Nvidia CUDA"}
    for i in range(len(docs_embeddings))
]

In [ ]:
print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

Data has 85 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 1536


Insert embeddings vector into the milvus db

In [ ]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)

{'insert_count': 85, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84], 'cost': 0}


In [ ]:
query_vectors = sentence_transformer_ef.encode_queries(["Tell me about CUDA programming model."])

res = client.search(
    collection_name = "demo_collection",  
    data = query_vectors,  
    limit = 3,  
    output_fields = ["text", "subject"],  
)

print(res[0])


[{'id': 18, 'distance': 0.7019416689872742, 'entity': {'text': 'This guide provides a detailed discussion of the CUDA programming model and programming interface. It then describes the hardware implementation, and provides guidance on how to achieve maximum performance. The appendices include a list of all CUDA-enabled devices, detailed description of all extensions to the C++ language, listings of supported mathematical functions, C++ features supported in host and device code, details on texture fetching, technical specifications of various devices, and concludes by introducing the low-level driver API.', 'subject': 'Nvidia CUDA'}}, {'id': 62, 'distance': 0.6892809271812439, 'entity': {'text': 'The CUPTI-API. The CUDA Profiling Tools Interface (CUPTI) enables the creation of profiling and tracing tools that target CUDA applications.', 'subject': 'Nvidia CUDA'}}, {'id': 66, 'distance': 0.6868547201156616, 'entity': {'text': 'vGPUs that support CUDA.', 'subject': 'Nvidia CUDA'}}]


In [ ]:
rel_res = []
for data in res[0]:
  print(data['entity']['text'])
  rel_res.append(data['entity']['text'])

This guide provides a detailed discussion of the CUDA programming model and programming interface. It then describes the hardware implementation, and provides guidance on how to achieve maximum performance. The appendices include a list of all CUDA-enabled devices, detailed description of all extensions to the C++ language, listings of supported mathematical functions, C++ features supported in host and device code, details on texture fetching, technical specifications of various devices, and concludes by introducing the low-level driver API.
The CUPTI-API. The CUDA Profiling Tools Interface (CUPTI) enables the creation of profiling and tracing tools that target CUDA applications.
vGPUs that support CUDA.


Now on retrieved relevant chunk I will perform reranking

In [ ]:
from pymilvus.model.reranker import CrossEncoderRerankFunction

ce_rf = CrossEncoderRerankFunction(
    model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",  # Specify the model name.
    device="cpu" # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
query = "Tell me about CUDA programming model."

results = ce_rf(
    query = query,
    documents = rel_res,
    top_k = 3,
)

final_result = ""
for result in results:
    print(f"Index: {result.index}")
    print(f"Score: {result.score:.6f}")
    print(f"Text: {result.text}\n")
    final_result += " "+ result.text


Index: 0
Score: 5.686930
Text: This guide provides a detailed discussion of the CUDA programming model and programming interface. It then describes the hardware implementation, and provides guidance on how to achieve maximum performance. The appendices include a list of all CUDA-enabled devices, detailed description of all extensions to the C++ language, listings of supported mathematical functions, C++ features supported in host and device code, details on texture fetching, technical specifications of various devices, and concludes by introducing the low-level driver API.

Index: 1
Score: -1.534823
Text: The CUPTI-API. The CUDA Profiling Tools Interface (CUPTI) enables the creation of profiling and tracing tools that target CUDA applications.

Index: 2
Score: -3.359596
Text: vGPUs that support CUDA.



Response Augmentation

In [ ]:
# do some prompt engineering to get the response align with user query

In [ ]:
!pip install accelerate
!pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating

query = "Tell me about CUDA programming model."

messages = [
    {
        "role": "system",
        "content": f"You are an intelligent assistant designed to provide accurate and relevant responses based on the given context: {final_result}. Generate a precise response.",
    },
    {"role": "user", "content": query},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.5, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]